In [1]:
!pip install livekit-api openai-whisper gradio gTTS diffusers transformers accelerate safetensors torch

import whisper
from livekit import api
import gradio as gr
from gtts import gTTS
import asyncio
import os
from diffusers import StableDiffusionPipeline
import torch

# ------------------ LiveKit Connection ------------------
LIVEKIT_URL = "https://genai-mq7kpfih.livekit.cloud"
LIVEKIT_API_KEY = "APIVZJeW43TvC4Q"
LIVEKIT_API_SECRET = "w2WWueQHqYu3xlbTwdjpuhzst0tf55zWtwtV8bHTq3A"

async def test_livekit():
    client = api.LiveKitAPI(
        LIVEKIT_URL,
        api_key=LIVEKIT_API_KEY,
        api_secret=LIVEKIT_API_SECRET
    )
    rooms = await client.room.list_rooms(api.ListRoomsRequest())
    print("✅ LiveKit connected. Rooms:", rooms)
    await client.aclose()

await test_livekit()

# ------------------ Whisper Model ------------------
model = whisper.load_model("base")

# ------------------ Stable Diffusion Turbo (FAST) ------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/sd-turbo",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)

# ------------------ Simple Rule-Based Chatbot ------------------
def generate_reply(user_text):
    text = user_text.lower()
    if "how are you" in text:
        return "I'm good, how about you?"
    elif "hello" in text or "hi" in text:
        return "Hello! Nice to meet you."
    elif "your name" in text:
        return "I’m your AI Voice Assistant connected with LiveKit."
    elif "bye" in text:
        return "Goodbye! Have a wonderful day."
    else:
        return "I heard you say: " + user_text

# ------------------ Voice Chat Pipeline ------------------
def voice_chat(audio_file):
    try:
        if audio_file is None:
            return "⚠️ No audio received", None, None

        # Step 1: Transcribe speech
        result = model.transcribe(str(audio_file))
        user_text = result["text"]

        # Step 2: Generate chatbot reply
        reply = generate_reply(user_text)

        # Step 3: Convert reply to speech
        reply_path = "reply.mp3"
        tts = gTTS(reply)
        tts.save(reply_path)

        # Step 4: Generate image ONLY if user asks
        image_path = None
        if any(word in user_text.lower() for word in ["draw", "generate", "create", "picture", "image"]):
            image = pipe(user_text, num_inference_steps=4).images[0]  # ⚡ super fast
            image_path = "generated.png"
            image.save(image_path)

        return f"🗣 You said: {user_text}\n🤖 Reply: {reply}", reply_path, image_path

    except Exception as e:
        return f"❌ Error: {str(e)}", None, None

# ------------------ Gradio UI ------------------
with gr.Blocks(css=".gradio-container {background: linear-gradient(to right, #fce4ec, #e3f2fd);} ") as demo:
    gr.Markdown("<h1 style='color:#1976d2;'>🎤 AI Voice + Image Assistant (LiveKit + Whisper + SD-Turbo)</h1>")
    gr.Markdown("Speak into the mic → AI will reply with text + voice 🎙 + generate an image if you ask 🎨")

    with gr.Row():
        inp = gr.Audio(sources=["microphone"], type="filepath", label="🎙 Speak here")

    with gr.Row():
        out_text = gr.Textbox(label="Chatbot Response", lines=3)
        out_audio = gr.Audio(label="🔊 AI Voice Reply")
        out_image = gr.Image(label="🎨 AI Generated Image")

    submit_btn = gr.Button("Submit 🎯")

    submit_btn.click(fn=voice_chat, inputs=inp, outputs=[out_text, out_audio, out_image])

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=99e186580e4517cf4f2126fabecbaf6deb966938813af6fa02b19f7f165b4453
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
✅ LiveKit connected. Rooms: 


100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 31.5MiB/s]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36e823d1818fc3f020.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
